# 準備

git cloneで今回利用するファイルを取得

In [1]:
!git clone https://github.com/labdemy-lectureteam/RAG-lecture-lesson1.git

Cloning into 'RAG-lecture-lesson1'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 56 (delta 17), reused 50 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 888.12 KiB | 9.45 MiB/s, done.
Resolving deltas: 100% (17/17), done.


必要パッケージのインストール

In [45]:
%pip install -qU langchain_community
%pip install -qU pypdf
%pip install -qU nltk
%pip install -qU langchain_openai langchain_chroma
%pip install -qU unstructured
%pip install -qU datasets
%pip install -qU ragas
%pip install -qU tqdm

API KEYの設定

In [18]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## RAG関連の関数の定義

streamlitアプリのrag.pyのコードをほとんどそのままコピペしてきてます。

In [39]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import shutil
import os
from langchain_openai import ChatOpenAI
import chromadb

from typing import List, Dict, Any
from langchain_core.vectorstores import VectorStore
from langchain_core.documents import Document

def load_PDF(path: str) -> List[Document]:
    if os.path.isdir(path):
        loader = PyPDFDirectoryLoader(path, glob="*.pdf")
        try:
            documents = loader.load()
        except Exception as error:
            print(error)
    elif os.path.isfile(path):
        if not path.lower().endswith('.pdf'):
            raise ValueError(f"与えられたファイル：  '{path}' はPDFではありません.")
        loader = PyPDFLoader(path)
        documents = loader.load()
    else:
        raise ValueError(f"与えられたパス： '{path}' はファイルでもディレクトリでもありません。")
    if not documents:
        raise ValueError(f"与えられたパス： '{path}' からのファイルの読み込みに失敗しました。")
    return documents

def create_chunks(documents: List[Document], chunk_size: int, chunk_overlap: int) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

def init_vector_db(embedding_model:Any,db_path:str)->Chroma:
    if os.path.exists(db_path):
        shutil.rmtree(db_path)
    chromadb.api.client.SharedSystemClient.clear_system_cache()
    vector_db = Chroma(
        collection_name='rag_app_collection',
        embedding_function=embedding_model,
        persist_directory=db_path
        )
    return vector_db

def get_context_from_db(vector_db:VectorStore, query:str, k:int=5, score_threshold:float=None)->List[Document]:
    contexts = vector_db.similarity_search_with_relevance_scores(query,
                                                                 k=k,
                                                                 score_threshold=score_threshold)
    return contexts

def format_prompt(contexts:List[Document], query:str, chat_history:List[Dict[str, str]])->str:
    PROMPT = """
    You are a helpful assistant. Answer the following questions based on the given context:
    chat history: {CHAT_HISTORY}

    context: {CONTEXT}

    Answer the following questions based on the given context:
    query: {QUERY}
    """
    prompt = ChatPromptTemplate.from_template(PROMPT)
    chat_history = '\n\n'.join([f"{message['role']}: {message['content']}" for message in chat_history])
    sources = [ {'source':doc[0].metadata['source'],'page':doc[0].metadata['page']} for doc in contexts ]
    contexts = '\n'.join([f"CONTEXT {idx}:\n{res.page_content}" for idx, (res, _score) in enumerate(contexts)])
    prompt = prompt.format(CHAT_HISTORY=chat_history,CONTEXT=contexts, QUERY=query)
    return prompt, sources

## ベクトル化

In [41]:
documents = load_PDF("/content/RAG-lecture-lesson1/sample_files")
chunks = create_chunks(documents=documents, chunk_size=100, chunk_overlap=0)
embedding_model= OpenAIEmbeddings(model='text-embedding-3-small')
vector_db = init_vector_db(embedding_model=embedding_model,db_path="database")
vector_db.add_documents(documents=chunks)

['eb42a0cd-39c0-4d1e-9f6d-60c646243c32',
 '68ee4356-f7c3-4c26-8ee1-3f054db291b5',
 'c6b5dd14-72e1-4f9b-955c-72243bcdc505',
 'abfbde24-3ca1-44f2-8d09-b86c71f3a6f9',
 '34848d46-bb88-45c0-9822-6c2cfcd1d08f',
 '78b22c02-af0d-4537-a51c-5d1105c3fe81',
 '87cdbccd-0cdd-481d-9fa7-a0bd4115f098',
 '48749863-e6dd-4d3a-b152-c7a4fd7dc33b',
 'c4963964-bd7b-4328-b3d1-bcc216b4fe0e',
 '489d16ab-1e02-4270-8e90-b46166dc0de5',
 '26523f49-1dc7-45e8-9f11-ae23efc707da',
 '93d2c94b-4ac3-4ce5-b94d-eeb7bb073168',
 '99ada330-b306-465d-9738-c88f20550106',
 '1af97ce3-33f5-47e7-a1c7-15c8592bb5da',
 'c67dcd2e-0375-4d6d-ba52-d1a2c08c65e4',
 '57f5f156-7e07-4881-b49d-4701a940b035',
 'c9136da7-bece-4577-85a3-0c715d0857d3',
 '0f4e149c-59a2-4b9b-914c-4a54a8a32017']

# RAGAsによるRAGの評価

## 質問とそれに対応する理想的な回答の用意

In [46]:
from datasets import Dataset

# 質問集
questions = ["この講義はどういうものですか？",
             "主催者はどういう団体ですか？",
             "合計何回の講義がありますか？",
             "講義日程はどうなってますか？",
             "参加したい場合はどうすれば良いですか？",
             "担当者の連絡先を教えてください",
             "参加費はいくらですか？",
             "何か事前準備は必要ですか？"]

# 理想的な回答
ideal_answers = ["この講義はRAGについて学ぶ講義です。",
                 "主催者はAI人材の育成に取り組んでいるLabdemyという団体です。",
                 "講義は合計4回です。",
                 "講義日程は：\n 第1回 2/2 (日) 14:00~15:30 \n 第2回 2/9 (日) 14:00~15:30 \n 第3回 2/16 (日) 14:00~15:30 \n 第4回 2/23 (日) 14:00~15:30 \nです。",
                 "参加したい場合はこちらの[リンク](https://discord.gg/JaQPZ5Gx?event=1323252933327847464)からdiscordサーバーに参加してください。",
                 "担当者の連絡先はlecture@labdemy.comです。",
                 "参加費は無料です。",
                 "事前準備はPython環境の準備、基礎的なプログラミング知識が必要です。"]

## 回答の生成

In [111]:
from langchain.chat_models import ChatOpenAI
from tqdm import tqdm

# モデルの準備
chat_model = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

# チャット履歴
chat_history = [] #これはずっと空っぽのまま

# コンテキスト情報
contexts = []

# LLMの回答
model_answers = []

# 各質問に対して回答を生成し、それを記録していく
for question in tqdm(questions):
  context = get_context_from_db(vector_db,question)
  final_prompt,_ = format_prompt(context, question,chat_history)
  response = chat_model.invoke(final_prompt)

  # contextから、文章のみを抽出して記録する
  context_text = [item[0].page_content for item in context]
  contexts.append(context_text)

  # モデルの回答を記録
  model_answers.append(response.content)

 12%|█▎        | 1/8 [00:06<00:47,  6.77s/it]<ipython-input-39-bd2c6b96f717>:55: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='26523f49-1dc7-45e8-9f11-ae23efc707da', metadata={'creationdate': '2024-12-30T12:27:58+00:00', 'creator': 'Chromium', 'moddate': '2024-12-30T12:27:58+00:00', 'page': 1, 'page_label': '2', 'producer': 'Skia/PDF m119', 'source': '/content/RAG-lecture-lesson1/sample_files/RAG講義のテスト受講案内.pdf', 'start_index': 451, 'total_pages': 3}, page_content='必要な準備： Python 環境の準備、基礎的なプログラミング知識\n参加費⽤：無料\n参加⽅法\nRAG 講義のテスト受講⽤ discord サーバーにご参加ください。（招待リンクは別途\n添付）'), 0.03670363463100257), (Document(id='489d16ab-1e02-4270-8e90-b46166dc0de5', metadata={'creationdate': '2024-12-30T12:27:58+00:00', 'creator': 'Chromium', 'moddate': '2024-12-30T12:27:58+00:00', 'page': 1, 'page_label': '2', 'producer': 'Skia/PDF m119', 'source': '/content/RAG-lecture-lesson1/sample_files/RAG講義のテスト受講案内.pdf', 'start_index': 351, 'total_pages': 3}, page_content='第 3 回  2/16 \ue081 ⽇ \

## RAGAs評価

In [112]:
#評価データセット作成

eval_dataset_dict = {
  "user_input": questions, #質問集
  "response": model_answers, #LLMの回答
  "retrieved_contexts": contexts, #コンテキスト情報
  "reference": ideal_answers #理想的な回答
}

eval_dataset = Dataset.from_dict(eval_dataset_dict)

In [113]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [114]:
scores = evaluate(
    dataset=eval_dataset,
    metrics=[context_precision,context_recall,answer_relevancy,faithfulness]
)

print(scores)

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[10]: AuthenticationError(Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************YuQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}})
ERROR:ragas.executor:Exception raised in Job[22]: AuthenticationError(Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************YuQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}})
ERROR:ragas.executor:Exception raised in Job[26]: AuthenticationError(Error code: 

{'context_precision': 0.6667, 'context_recall': 0.8125, 'answer_relevancy': 0.8210, 'faithfulness': 0.8281}


In [122]:
for model_answer,ideal_answer in zip(model_answers,ideal_answers):
  print("="*20)
  print("【LLMの回答】\n",model_answer)
  print("\n")
  print("【理想的な回答】\n",ideal_answer)

【LLMの回答】
 Assistant: 
この講義は、Retrieval-Augmented Generation（RAG）について網羅的に学ぶものです。合計4回の講義を通じて、RAGについて深く理解することが目指されています。また、この講義はオンラインで行われ、Zoomリンクを通じて参加することができます。


【理想的な回答】
 この講義はRAGについて学ぶ講義です。
【LLMの回答】
 Assistant: 
主催者は「Labdemy」です。


【理想的な回答】
 主催者はAI人材の育成に取り組んでいるLabdemyという団体です。
【LLMの回答】
 Assistant: 合計で4回の講義があります。


【理想的な回答】
 講義は合計4回です。
【LLMの回答】
 Assistant: 
講義は次の日程で行われます：
第 3 回は2月16日、14:00から15:30まで。
第 4 回は2月23日、14:00から15:30までです。全ての講義はオンライン（Zoom）で行われます。


【理想的な回答】
 講義日程は：
 第1回 2/2 (日) 14:00~15:30 
 第2回 2/9 (日) 14:00~15:30 
 第3回 2/16 (日) 14:00~15:30 
 第4回 2/23 (日) 14:00~15:30 
です。
【LLMの回答】
 Assistant: 
参加方法は次のとおりです。まず、Python環境の準備と基礎的なプログラミング知識が必要です。次に、Zoomリンクを通じてオンラインで講義に参加します。そのリンクは後日共有されます。また、RAG講義のテスト受講用のDiscordサーバーにも参加する必要があります。招待リンクは別途添付されます。参加費用は無料です。


【理想的な回答】
 参加したい場合はこちらの[リンク](https://discord.gg/JaQPZ5Gx?event=1323252933327847464)からdiscordサーバーに参加してください。
【LLMの回答】
 Assistant: 
担当者への連絡は、lecture@labdemy.com 宛にご連絡ください。


【理想的な回答】
 担当者の連絡先はlecture@labdemy.comです。
【LLMの回答】
 Assistant: 参加費は